<a href="https://colab.research.google.com/github/DerNiccoo/TheEarlyBird/blob/main/Aufgabe3/Alina/Assignment3_Notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [3]:
#Choose a device: GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# **Define new Dataset as Subset from CIFAR10**

In [4]:
class CIFAR_X(torchvision.datasets.CIFAR10):

    def __init__(self, *args, exclude_list=[], **kwargs):
        super(CIFAR_X, self).__init__(*args, **kwargs)

        if exclude_list == []:
            return

        labels = np.array(self.targets)
        exclude = np.array(exclude_list).reshape(1, -1)
        filter = ~(labels.reshape(-1, 1) == exclude).any(axis=1)
        
        self.data = self.data[filter]
        self.targets = labels[filter].tolist()

# **Initalizing the new defined trainset and trainloader**

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = CIFAR_X(root='./data', train=True, download=True, transform = transform, exclude_list=[8,9])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse')

Extracting ./data/cifar-10-python.tar.gz to ./data


# **Define the Network**

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 8)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# **Train Method**

In [7]:
def train(net, epochs, criterion, optimizer):
  
  for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Finished Training


# **Initializing the Network with parameters**

In [8]:
import time
t0 = time.time()

net = Net()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
epochs = 2

train(net, epochs, criterion, optimizer)
print('{} seconds'.format(time.time() - t0))

[1,  2000] loss: 1.998
[1,  4000] loss: 1.734
[1,  6000] loss: 1.545
[1,  8000] loss: 1.440
[1, 10000] loss: 1.396
[2,  2000] loss: 1.321
[2,  4000] loss: 1.275
[2,  6000] loss: 1.296
[2,  8000] loss: 1.265
[2, 10000] loss: 1.236
100.01767110824585 seconds


# **Safe the trained Net**

In [18]:
PATH = '/content/drive/My Drive/KI_Lab/model'
torch.save(net.state_dict(), PATH)